In [ ]:
import os
import sys
sys.path.append('../ph4ser/')

def reload_libs():
    """
    This allows you to reload the library file without restarting the kernel.
    
    Usage:
    In [1]: reload_libs()
    """
    import mlibs as mlibs
    from importlib import reload
    reload(mlibs)
    import ph4ser as ph4
    reload(ph4)

In [ ]:
config = ph4.Configuration()
config.cell_size = None
config.receiver = None
config.imsize = int(2.5*1024)
config.imsizey = int(2.5*1024)
steps = [
    'startup',          # create directory structure, start variables and clear visibilities.
#     'fov_image',      # create a FOV image
    'save_init_flags',  # save (or restore) the initial flags
    'statwt',           # run statwt on the initial data
    'autoflag_init',    # run rflag on the initial data -- use for L and S band data.
    'test_image',       # create a test image
    'select_refant',    # select reference antenna
    'p0',               # initial test  of selfcal, phase only (p)
    'p1',               # redo phase-only selfcal (if enough flux density); ignores p0
    'p2',               # continue phase-only selfcal (incremental)
    'ap1',              # amp-selfcal (ap); uses p0 or (p1 and p2)
    'split_trial_1',    # split the data after first trial (and run wsclean)
    # 'autoflag_final',   # run rflag on the final data (use for L and S band data)
    'report_results'    # report results of first trial
]

In [ ]:
pipeline = ph4.Pipeline(config)

In [ ]:
config.path = "/media/sagauga/galnet/LIRGI_Sample/VLA-Archive/A_config/23A-324/C_band/MCG+12-02-001/autoselfcal/"
config.field = "MCG12-02-001"
config.vis_name = "MCG12-02-001.calibrated.avg"
config.savename = "_A_C_sf"
config.visibility_info = {'path':config.path,
                          'field':config.field,
                          'vis_name':config.vis_name,
                          'savename':config.savename
                         }

print(config.visibility_info)

In [ ]:
pipeline.run_step('startup')
config.cell_size = pipeline.get_cell_size(pipeline.g_vis)
print(f"Cell size = {config.cell_size}")
config.receiver = pipeline.check_band(pipeline.g_vis)
print(f"Receiver = {config.receiver}")

In [ ]:
if 'save_init_flags' in config.steps:
    pipeline.run_step('save_init_flags')
if 'statwt' in config.steps:
    pipeline.run_step('statwt')
if 'autoflag_init' in config.steps:
    pipeline.run_step('autoflag_init')

In [ ]:
if 'test_image' in config.steps:
    pipeline.run_step('test_image')
pipeline.check_init_parameters()

In [ ]:
if 'select_refant' in config.steps:
    pipeline.run_step('select_refant')

if 'p0' in config.steps:
    pipeline.run_step('p0')
pipeline.check_p0_parameters()

In [ ]:
if (('p1' in config.steps) and ('p1' not in pipeline.steps_performed) and
        ('p1' in pipeline.parameter_selection['p0_pos'])):
    pipeline.run_step('p1')

In [ ]:
if (('p2' in config.steps)
        and ('p2' not in pipeline.steps_performed)
        and ('p2' in pipeline.parameter_selection['p0_pos'])):
    pipeline.run_step('p2')

In [ ]:
if (('ap1' in config.steps) and
        ('ap1' not in pipeline.steps_performed) and
        ('ap1' in pipeline.parameter_selection['p0_pos'].keys())):
    pipeline.run_step('ap1')

In [ ]:
if 'split_trial_1' in config.steps:
    pipeline.run_step('split_trial_1')

# if 'autoflag_final' in config.steps:
#     pipeline.run_step('autoflag_final')

if 'report_results' in config.steps:
    pipeline.run_step('report_results')